# Project 1: Gene List Enrichment (Beginner Friendly)

**Goal:** Given a small list of human gene symbols, run pathway/GO enrichment and make a simple plot.

**What you'll learn:**
- How to work with lists of genes in Python
- How to run enrichment using `gseapy` (Enrichr)
- How to visualize top pathways
- How to export results to CSV

> This notebook uses the **Enrichr** API via `gseapy`. Your computer will need an **internet connection** when you run the enrichment step.


## 0) Environment setup (run in your terminal, not inside the notebook)

If you don't have a Python environment yet, the simplest is to use one of these options:

### Option A: Using `pip` directly
```bash
python -m venv bfx-env
source bfx-env/bin/activate      # on Windows: bfx-env\Scripts\activate
pip install --upgrade pip
pip install pandas matplotlib gseapy mygene
```

### Option B: Using conda (Anaconda/Miniconda)
```bash
conda create -n bfx-env python=3.10 -y
conda activate bfx-env
pip install pandas matplotlib gseapy mygene
```

Then open this notebook in Jupyter Lab/Notebook and proceed.

## 1) Imports
If any import fails, run the `pip install ...` commands above in your terminal.

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

try:
    import gseapy as gp
    GSEAPY_OK = True
except Exception as e:
    print("gseapy not available:", e)
    GSEAPY_OK = False

try:
    from mygene import MyGeneInfo
    MYGENE_OK = True
except Exception as e:
    print("mygene not available:", e)
    MYGENE_OK = False

print("gseapy:", GSEAPY_OK, "| mygene:", MYGENE_OK)

## 2) Provide a small gene list
You can paste your own list here. We'll start with a toy list enriched for **cell cycle** so you can see obvious hits.

In [ ]:
genes = [
    "MKI67","CCNB1","CCNB2","CDK1","BIRC5","TOP2A","PCNA","PLK1","CDC20","AURKA",
    "AURKB","KIF11","KIF2C","KIF20A","CHEK1","CHEK2","TYMS","CENPF","CENPA","UBE2C"
]
len(genes), genes[:5]

## 3) (Optional) Validate gene symbols with MyGene
This step checks if your symbols are valid **HGNC** human symbols and maps to Entrez/Ensembl IDs. Requires internet.

In [ ]:
if MYGENE_OK:
    mg = MyGeneInfo()
    q = mg.querymany(genes, scopes='symbol', fields='symbol,name,entrezgene,ensembl.gene', species='human')
    df_check = pd.DataFrame(q)
    display(df_check[['query','symbol','name','entrezgene']].head(10))
else:
    print("mygene not installed — skipping symbol validation.")

## 4) Run enrichment with Enrichr via gseapy
We'll try a few popular libraries. You can change them later by running `gp.get_library_name()`.

In [ ]:
if not GSEAPY_OK:
    raise SystemExit("gseapy is required for this step. Please install: pip install gseapy")

gene_sets = [
    'KEGG_2021_Human',
    'Reactome_2022',
    'GO_Biological_Process_2021'
]

outdir = 'enrichr_out'
os.makedirs(outdir, exist_ok=True)

enr = gp.enrichr(
    gene_list=genes,
    description='demo_enrichment',
    gene_sets=gene_sets,
    outdir=outdir,
    cutoff=0.05  # adjusted p-value threshold
)
res = enr.results  # or enr.res2d in some versions
res.head()

## 5) Visualize top terms
We'll draw a simple bar plot of the **top 10** enriched terms by adjusted p-value.

In [ ]:
if res is None or len(res)==0:
    print("No significant terms found at the chosen cutoff. Try increasing cutoff or changing libraries.")
else:
    # Sort by adjusted p-value
    res_sorted = res.sort_values(by=['Adjusted P-value','P-value'], ascending=[True, True]).head(10).copy()
    res_sorted['-log10(AdjP)'] = -res_sorted['Adjusted P-value'].apply(lambda x: (0 if x<=0 else __import__('math').log10(x)))
    # The above creates negative values; flip sign for plotting clarity
    res_sorted['poslog10'] = res_sorted['-log10(AdjP)'] * -1
    
    plt.figure(figsize=(8,5))
    plt.barh(res_sorted['Term'], res_sorted['poslog10'])
    plt.xlabel('-log10(Adjusted P-value)')
    plt.ylabel('Term')
    plt.title('Top Enriched Terms')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()

## 6) Save results to CSV
You'll get one CSV with all results. You can also filter by a single library if you like.

In [ ]:
ts = datetime.now().strftime('%Y%m%d_%H%M%S')
csv_path = os.path.join(outdir, f'enrichr_results_{ts}.csv')
if res is not None:
    res.to_csv(csv_path, index=False)
    print('Saved:', csv_path)
else:
    print('No results to save.')

## 7) (Optional) Quick interpretation helper
This cell prints a tiny summary you can paste into a report.

In [ ]:
if res is not None and len(res):
    top = res.sort_values(by=['Adjusted P-value','P-value']).head(5)
    summary_lines = [
        f"Analyzed {len(genes)} genes. Top enriched terms (adj p-value):"
    ]
    for _, row in top.iterrows():
        summary_lines.append(f" - {row['Term']} (AdjP={row['Adjusted P-value']:.2e})")
    print("\n".join(summary_lines))
else:
    print("No significant enrichment to summarize.")

---
### Next steps (when you feel ready)
- Replace the toy gene list with your own set (e.g., top 100 upregulated genes from an RNA-seq study).
- Try different libraries: `gp.get_library_name()` shows available Enrichr libraries.
- Export a publication-style figure (higher DPI) and add text interpretation.
- Wrap the enrichment into a small Python function or CLI tool.
